# [01] Cross Entropy in Cartpole 

### Imports & Constants

In [1]:
import os
import datetime
import numpy as np
from collections import namedtuple

import gym

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

<br> 

### Neural Net

In [2]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

<br> 

### Helper Functions

In [3]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def sample_episodes(env, network, batch_size):
    '''Retrives a batch of episodes ='''
    
    episodes = [] 
    episode_steps = [] 
    episode_reward = 0.0
    softmax = nn.Softmax(dim=1)
    observation = env.reset()
    
    while True:
        
        # Convert observation into tensor 
        observation_t = torch.FloatTensor([observation])
        
        # Get action via NN and random sampling 
        action_probs_t = softmax(network(observation_t))
        action_probs = action_probs_t.data.numpy()[0]
        action = np.random.choice(len(action_probs), p=action_probs)
        
        # Take step 
        next_observation, reward, is_done, _ = env.step(action)
        
        # Accumulate reward 
        episode_reward += reward
        
        # Save step 
        step = EpisodeStep(observation=observation, action=action)
        episode_steps.append(step)
        
        if is_done:
            
            # Save episode 
            episode = Episode(reward=episode_reward, steps=episode_steps)
            episodes.append(episode)
            
            # Reset 
            episode_reward = 0.0
            episode_steps = []
            next_observation = env.reset()
            
            if len(episodes) == batch_size:
                
                # Return episodes when we've accumulated enough  
                yield episodes
                episodes = []
        
        observation = next_observation


def filter_episodes(episodes, percentile):
    '''Extracts the top/elite episodes'''
    
    # Extract rewards from episodes 
    rewards = list(map(lambda s: s.reward, episodes))
    
    # Calculate reward boundary & mean
    reward_boundary = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    # Save the "elite" episodes 
    train_observations = []
    train_actions = []
    for reward, steps in episodes:
        
        if reward < reward_boundary:
            continue
        
        train_observations.extend(map(lambda step: step.observation, steps))
        train_actions.extend(map(lambda step: step.action, steps))

    # Convert to tensors 
    train_observations_t = torch.FloatTensor(train_observations)
    train_actions_t = torch.LongTensor(train_actions)
    
    return train_observations_t, \
           train_actions_t, \
           reward_boundary, \
           reward_mean

<br> 

### Main 

In [10]:
# Create environment 
env = gym.make("CartPole-v0")
#env = gym.wrappers.Monitor(env, "./recordings/01_recording", force=True)
recording = gym.wrappers.monitoring.video_recorder.VideoRecorder(env, base_path="./recordings/01_recording")

# Observation & action space size 
observation_size = env.observation_space.shape[0]
num_actions = env.action_space.n

# Network, Loss, Optimizer 
network = Net(observation_size, HIDDEN_SIZE, num_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=network.parameters(), lr=0.01)

# Log directory for TensorBoard
log_dir = os.path.join('./runs/01_runs', datetime.datetime.now().strftime("%Y:%m:%d-%H:%M:%S"))
writer = SummaryWriter(log_dir)
print('(Saving to ' + log_dir + ')\n')

# Training 
for epoch, batch in enumerate(sample_episodes(env, network, BATCH_SIZE)):

    recording.capture_frame()
    
    observations_t, actions_t, reward_boundary, reward_mean = filter_episodes(batch, PERCENTILE)
        
    optimizer.zero_grad()
    action_scores_t = network(observations_t)
    loss_t = objective(action_scores_t, actions_t)
    loss_t.backward()
    optimizer.step()
    
    print("%d: loss=%.3f, reward_mean=%.1f, reward_boundary=%.1f" % (epoch, loss_t.item(), reward_mean, reward_boundary))
    writer.add_scalar("Loss", loss_t.item(), epoch)
    writer.add_scalar("Reward Boundary", reward_boundary, epoch)
    writer.add_scalar("Reward Mean", reward_mean, epoch)
        
    if reward_mean > 199:
        print("Solved!")
        break
    
writer.close()
recording.close()

(Saving to ./runs/01_runs/2020:09:24-08:58:47)

0: loss=0.682, reward_mean=21.1, reward_boundary=23.0
1: loss=0.653, reward_mean=23.2, reward_boundary=22.5
2: loss=0.667, reward_mean=18.2, reward_boundary=19.5
3: loss=0.660, reward_mean=19.8, reward_boundary=20.0
4: loss=0.677, reward_mean=24.4, reward_boundary=30.0
5: loss=0.657, reward_mean=29.4, reward_boundary=31.0
6: loss=0.652, reward_mean=21.5, reward_boundary=26.5
7: loss=0.639, reward_mean=25.5, reward_boundary=19.5
8: loss=0.633, reward_mean=21.4, reward_boundary=24.5
9: loss=0.626, reward_mean=25.5, reward_boundary=28.0
10: loss=0.616, reward_mean=28.6, reward_boundary=32.0
11: loss=0.608, reward_mean=30.9, reward_boundary=33.0
12: loss=0.623, reward_mean=38.4, reward_boundary=45.0
13: loss=0.653, reward_mean=49.3, reward_boundary=59.0
14: loss=0.603, reward_mean=42.8, reward_boundary=42.0
15: loss=0.610, reward_mean=58.8, reward_boundary=68.0
16: loss=0.620, reward_mean=48.6, reward_boundary=47.5
17: loss=0.619, reward_mean

<br> 

### Play Video

In [8]:
from IPython.display import Video
Video("./recordings/01_recording.mp4", width=600)

<br> 

## TensorBoard

In [ ]:
# Upload all experiments 
!tensorboard dev upload --logdir='./runs/01_runs' --name='01_cross_entropy_cartpole'

In [ ]:
# List all experiments 
!tensorboard dev list

<br>